In [1]:
!pip install redis

   ---------------------------------------- 0.0/252.0 kB ? eta -:--:--
   - -------------------------------------- 10.2/252.0 kB ? eta -:--:--
   ---- ---------------------------------- 30.7/252.0 kB 640.0 kB/s eta 0:00:01
   --------- ----------------------------- 61.4/252.0 kB 648.1 kB/s eta 0:00:01
   -------------- ------------------------ 92.2/252.0 kB 581.0 kB/s eta 0:00:01
   ------------------ ------------------- 122.9/252.0 kB 599.1 kB/s eta 0:00:01
   -------------------------- ----------- 174.1/252.0 kB 697.2 kB/s eta 0:00:01
   --------------------------------- ---- 225.3/252.0 kB 762.6 kB/s eta 0:00:01
   -------------------------------------  245.8/252.0 kB 793.0 kB/s eta 0:00:01
   -------------------------------------  245.8/252.0 kB 793.0 kB/s eta 0:00:01
   -------------------------------------  245.8/252.0 kB 793.0 kB/s eta 0:00:01
   -------------------------------------  245.8/252.0 kB 793.0 kB/s eta 0:00:01
   -------------------------------------  245.8/252.0 kB 

In [1]:
import redis
import pandas as pd
import numpy as np

from __future__ import annotations

import time
from concurrent.futures import ThreadPoolExecutor, as_completed
from typing import Generator
from tqdm import tqdm

In [4]:
rd = redis.Redis(host='ec2-54-167-24-43.compute-1.amazonaws.com', charset="utf-8", decode_responses=True, password='redis')

In [5]:
rd.ping()

True

In [8]:
researches_df = pd.read_csv("../data/merged_researches.csv", index_col=0)
authors_df = pd.read_csv("../data/merged_authors.csv", index_col=0)
affiliations_df = pd.read_csv("../data/merged_affiliations.csv", index_col=0)

In [13]:
researches_df = researches_df.fillna("")
authors_df = authors_df.fillna("")
affiliations_df = affiliations_df.fillna("")

In [23]:
researches_list = researches_df.to_dict(orient='records')
authors_list = authors_df.to_dict(orient='records')
affiliations_list = affiliations_df.to_dict(orient='records')

In [61]:
def push_to_redis(key, data):
    rd.hset(f"{key}:{data['id']}", mapping=data)

def multithread_push_redis(key, items):
    with tqdm(total=len(items)) as pbar:
        with ThreadPoolExecutor(max_workers=64) as executor:
            futures = [executor.submit(push_to_redis, key, item) for item in items]
            for future in as_completed(futures):
                pbar.update(1)

In [62]:
multithread_push_redis("research", researches_list)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 45167/45167 [03:45<00:00, 200.26it/s]


In [63]:
multithread_push_redis("author", authors_list)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 197709/197709 [14:55<00:00, 220.86it/s]


In [64]:
multithread_push_redis("affiliation", affiliations_list)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30895/30895 [02:19<00:00, 221.36it/s]


In [ ]:
def pull_to_redis(data):
    result = rd.hgetall(data)
    return result
    

def multithread_pull_redis(items):
    results = []
    with tqdm(total=len(items)) as pbar:
        with ThreadPoolExecutor(max_workers=64) as executor:
            futures = [executor.submit(pull_to_redis, item) for item in items]
            for future in as_completed(futures):
                pbar.update(1)
                results.append(future.result())
    return results
    
                
                

In [7]:
research_data = rd.keys('research:*')

In [ ]:
research_results = multithread_pull_redis(research_data)

  0%|                                                                                        | 0/45157 [00:00<?, ?it/s]

research:85123694402
research:84963635521
research:85138055986
research:85015364808
research:85127604051
research:85147996710
research:85121303516
research:84953276697
research:85079549089
research:85118344086
research:85118336295
research:85130972397
research:85103596399
research:85112733805
research:85056695606
research:85072103422
research:85059833342
research:85123096926
research:85047745274
research:85057738318
research:85075978640
research:84956927264
research:85069662735
research:85151652714
research:84953278136
research:85183927503
research:85040196313
research:85058197269
research:85019148722
research:84957085675
research:85113437029
research:85092773584
research:85122595988
research:85077316479
research:85089096954
research:85102311275
research:85099920579
research:85084169252
research:85140932374
research:85052927013
research:85145424785
research:85089660939
research:85098941464
research:85091699096
research:85099752938
research:85123207956
research:85069942181
research:8506

  0%|                                                                             | 1/45157 [00:02<34:17:12,  2.73s/it]

research:85054454626
research:85147120167
research:85121299978
research:85122159054
research:85145230883
research:85066477848
research:85147892510
research:85127925757
research:85128731630
research:85115216237
research:85147600022
research:85099158973
research:85121528797
research:85149913273
research:85064790310
research:85124281035
research:85146395639
research:85166653319
research:85141782467
research:85148374353
research:85046277853
research:85113262462
research:84919428199
research:85075160347
research:85146321258
research:85093952268
research:84934932558
research:85181179193
research:85091767244
research:85121743258
research:85148388741
research:85019777295
research:85102688888
research:85188253860
research:85072528479
research:85132152323
research:85078202253
research:85100156646
research:85077968640
research:85072766903
research:85119662503
research:85076947669
research:85105391010
research:85022079606
research:85149918215
research:84958641905
research:84964058484
research:8516

  0%|▏                                                                             | 113/45157 [00:02<14:07, 53.14it/s]

...

research:85080989945research:85102581218
research:85047764434
research:85105961631
research:85089233076
research:85074523430
research:85047624518
research:85026442793
research:85111390577
research:85091404259
research:84949623855
research:85142627329
research:85061507707
research:85055145038
research:85142744005
research:85144259935
research:85112250652
research:85149687513
research:85081674112
research:84921822776
research:85109076739
research:85108575078
research:85164661935
research:85127572171
research:85087769195
research:85168409824
research:85025173870
research:84920500606

research:85146663372
research:85067491861
research:84985995746
research:84920973777
research:85107616347
research:85060027156
research:85097759056
research:85102622106
research:85077696106
research:84964757040
research:85123852827
research:85040719852
research:84952327648
research:85011964405
research:85074280433
research:85122021738
research:85169126747
research:85142904892
research:85079130561
research:8514

  2%|█▋                                                                           | 956/45157 [00:09<03:15, 226.58it/s]

research:85122444845
research:85168262758
research:84921417526
research:85122888816
research:85092373590
research:85092073308
research:85035335791
research:85181477615
research:85150547923
research:85028547706
research:85102157623
research:85084151441
research:85117098397
research:85125051005
research:85063126901
research:85053470598
research:85149013993
research:85090249624
research:85123904963
research:85085258157
research:85040599708
research:85144015223
research:85020265665
research:85077263617
research:85098938732
research:85104640674
research:85174444128
research:85059475911
research:85092496038
research:85049211373
research:84959078550
research:85111770685
research:84920097199
research:85077319110
research:85076902487
research:85076144157
research:85143949293
research:85069596524
research:85106245139
research:85167455063
research:85059247015
research:85098284761
research:85062474773
research:85039731551
research:85024367012
research:85097876396
research:85073496257
research:8503

  2%|█▋                                                                           | 963/45157 [00:24<03:15, 226.58it/s]

research:85122863585
research:85044321988
research:85150349836
research:85181632669
research:85107555192
research:85136124510
research:85006791371
research:85071968224
research:85027927068
research:85137693931
research:85145429628
research:85010619018
research:85075986792
research:85117577977
research:85144014768
research:84921731045
research:85148660834
research:85114099713
research:85140376438
research:84909645093
research:85009892136
research:85121715892
research:85099237251
research:85122429288
research:85028534997
research:85106478859
research:85054928855
research:85099172784
research:85162212043
research:85073251065
research:85084362531
research:85111105011
research:85066076515
research:85121134357
research:85101031274
research:85098960322
research:85026258606
research:85073444905
research:85113773226
research:85140016893
research:85158914370
research:85064257036
research:85129108577
research:85055027807
research:85047419035
research:85063498699
research:84969921246
research:8506

In [17]:
research_results[0:10]

[{}, {}, {}, {}, {}, {}, {}, {}, {}, {}]